In [7]:
#pip install todoist-python
from todoist import TodoistAPI, models
import pandas as pd
import numpy as np
from dotenv import load_dotenv, dotenv_values
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [31]:
credentials = dotenv_values('../../usage/.env')

In [11]:
api = TodoistAPI(credentials["TOKEN"])
s = api.sync()

In [141]:
def get_todoist_activities(api, pages, limit=100):
    activities = pd.DataFrame()
    for p in range(pages):
        if p > 0:
            activities = all_activities
        activities_on_page_df = pd.DataFrame(api.activity.get(page=p, limit=100)['events'])                                                                       
        all_activities = pd.concat([activities, activities_on_page_df])
    return all_activities

def get_parent_id(api, object_id):
    object_id = str(object_id)
    info_list = [api.items.get_by_id(object_id)] 
    for item in info_list:
        try:
            if ('item' in item.keys()) and ('parent_id' in item['item'].keys()):
                return item['item']['parent_id']
        except AttributeError:
            return None

In [142]:
df = get_todoist_activities(api, pages=5, limit=100)

In [145]:
df['content'] = df['extra_data'].apply(lambda x: x['content'] if 'content' in x.keys() else None)
df['due_date'] = df['extra_data'].apply(lambda x: x['due_date'] if 'due_date' in x.keys() else None)
df['name'] = df['extra_data'].apply(lambda x: x['name'] if 'name' in x.keys() else None)
object_id_list = [get_parent_id(api, idd) for idd in df['object_id'].unique()]
parent_id_dp = pd.DataFrame([object_id_list, df['object_id'].unique()]).T
parent_id_dp = parent_id_dp.rename(columns={0:'parent_id', 1: 'object_id'}).astype(int, errors='ignore')
df = df.merge(parent_id_dp, on='object_id')

In [146]:
df['event_date'].min(), df['event_date'].max()

('2022-03-14T12:25:37Z', '2022-04-14T23:05:58Z')

In [147]:
df.head(5)

,event_date,event_type,extra_data,id,initiator_id,object_id,object_type,parent_item_id,parent_project_id,content,due_date,name,parent_id_x,parent_id_y
0,2022-04-14T23:05:58Z,added,"{'client': 'Todoist-Android/v9498', 'content':...",15621758154,NaN,5766426657,item,NaN,2274628276,Fazer a barba,None,None,NaN,NaN
1,2022-04-14T22:09:38Z,updated,"{'client': 'Todoist-Android/v9498', 'content':...",15621346696,NaN,5523868582,item,NaN,2274628276,Passear com Nina,2022-04-19T02:59:59Z,None,NaN,NaN
2,2022-04-14T00:18:27Z,updated,"{'client': 'Mozilla/5.0; Todoist/1996', 'conte...",15608713648,NaN,5523868582,item,NaN,2274628276,Passear com Nina,2022-04-15T02:59:59Z,None,NaN,NaN
3,2022-04-06T23:34:39Z,completed,"{'client': 'Mozilla/5.0; Todoist/1989', 'conte...",15518256016,NaN,5523868582,item,NaN,2274628276,Passear com Nina,None,None,NaN,NaN
4,2022-03-30T23:47:00Z,completed,"{'client': 'Mozilla/5.0; Todoist/1979', 'conte...",15427284962,NaN,5523868582,item,NaN,2274628276,Passear com Nina,None,None,NaN,NaN


In [151]:
# Pega o arquivo com name enddate mais rescente
from glob import glob
import os
from datetime import date, datetime, timedelta
path_data = "../../usage/data/"
glob_search = f"{path_data}*.csv"

max_dates_on_folder = [
    d.split("/")[-1].split('-')[-1].split('.')[0] for d in glob(glob_search)
]
def get_min_diff_date_from_today(list_of_dates):
    diff_dates = {
        datetime.today() - datetime.strptime(d,"%Y%m%d" ):d 
        for d in list_of_dates
    }
    min_diff = min(list(diff_dates.keys()))
    return diff_dates[min_diff]
    
current_state_date = get_min_diff_date_from_today(max_dates_on_folder)

# filtra os registros que derem match com a data

df['event_date_formated'] = pd.to_datetime(
    df['event_date'], infer_datetime_format=True
).dt.strftime("%Y%m%d" )
df_new = df[df['event_date_formated'] >= current_state_date]
dates_to_dave_file = df_new['event_date_formated'].min(), df_new['event_date_formated'].max()

file_name = f"TODOIST-{dates_to_dave_file[0]}-{dates_to_dave_file[1]}.csv"

print(file_name)

df_new = df_new.drop('event_date_formated', axis=1)
#Salva novo df


TODOIST-20220409-20220414.csv


In [152]:
df_new.to_csv(os.path.join(path_data,file_name), index=False)